# Local PKI with OpenSSL

This guide assumes of _you_:

* Familiarity with Linux command line, bash commands, and basic CLI navigation

* That you have either Windows with WSL2 / MSYS, or a Linux machine/VM 

* That you intend to create and label certs for maximum clarity and auditability

This guide assumes of the _CA design_:

* **Each service gets its own private key + certificate**

* All certs are signed by one local root CA

* No wildcard certs

* No shared keys

## One Certificate per Service (Best‑Practice Guide)

This guide walks through creating and operating a local Certificate Authority (CA) and issuing individual TLS certificates per service using OpenSSL.

This approach is:

* Explicit

* Auditable

* Easy to rotate

Which is ideal for homelabs, internal services, airgap environments, and Docker stacks or K8s/K3s clusters which need cryptographic transmission protection, but don't need a full CA stack for whatever reason.

## What's in it for me?

This Python notebook gives you:

* Clear cert ownership per service

* Easy revocation and rotation of local certificates

* Minimal blast radius if you make a mistake

It’s the same mental model used in real PKI systems. It's just scaled down to a homelab where a busy person doesn't have all day to run an AD CA with all the gubbins.

Instead, you can create this, one-n-done, and use it for everything. This notebook's code cells can be edited and run *ad infinitum, ad nauseam*. 🤓💻

## 0) What you're building with this notebook:

For each service:

* One private key (never shared)

* One certificate (signed by your CA)

* One hostname (with SANs)

Shared across everything:

* One root CA

* Trusted once by browsers and clients

Each service is self‑contained.

# Recommended example structure:
```
~/pki/
├── ca/
│   ├── ca.key
│   ├── ca.crt
│   └── ca.srl
└── services/
    ├── n8n.local.test/
    │   ├── server.key
    │   ├── server.csr
    │   ├── server.crt
    │   └── server.cnf
    └── grafana.local.test/
        ├── server.key
        ├── server.csr
        ├── server.crt
        └── server.cnf
    ... and so on, so forth
```

In [ ]:
%%bash
## 1) Create a PKI Workspace
#!/usr/bin/env bash
set -euo pipefail
echo "Creating PKI workspace..."
mkdir -p ./pki/{ca,services}

## 2) Create the Root Certificate Authority (One‑Time)

### 2.1 Generate the CA private key

In [ ]:
%%bash
#!/usr/bin/env bash
set -euo pipefail
cd ./pki/ca
echo "Generating CA private key..."
openssl genrsa -out ca.key 4096

### 2.2 Create the CA certificate

Pay attention to the `-subj` line. `C=` is "Country", `ST` is State, `L` is Local, `O` is for the Organization, and the `CN` is the Container Name. Which is to say, the thing, within the Organization, at the Local (building, site, whatever), in the State, in the Country, that the cryptographic certificate authority is tied to.

This is one of the ways that cryptographic certificates can encode a bunch of information that's easily read by humans.

**Security note:**
Treat `ca.key` like a password vault. Anyone with it can mint trusted certs. You should additionally encrypt and secure `ca.key` through other means, and ensure you have an offsite (read: not on your computer, or in your house) backup of both the key itself, as well as whatever additional crypto or tools you use to encrypt/decrypt the key, so that you can recover it even if the computer you're reading this on ceased to exist tomorrow.

In [ ]:
%%bash
#!/usr/bin/env bash
set -euo pipefail
openssl req -x509 -new -nodes \
  -key ./pki/ca/ca.key \
  -sha256 -days 3650 \
  -out ./pki/ca/ca.crt \
  -subj "/C=COUNTRY_HERE/ST=STATE_OR_PROVINCE_HERE/L=Local/O=Homelab/CN=Python Notebook Root CA"
echo "Created CA certificate: ./pki/ca/ca.crt"
cat ./pki/ca/ca.crt

## 3) Issue a Certificate for a Service

Example service: `n8n.local.test`

Section 3 will walk you through the creation of HTTPS certificates to use with the self-hostable automation tool, `n8n`.

In [ ]:
%%bash
### 3.1 Create a service directory
#!/usr/bin/env bash
set -euo pipefail
echo "Creating n8n.local.test service directory..."
mkdir -p services/n8n.local.test
echo "Created directory: services/n8n.local.test"

In [ ]:
%%bash
### 3.3 Generate the service private key
#!/usr/bin/env bash
set -euo pipefail
echo "Generating private key for n8n.local.test..."
openssl genrsa -out services/n8n.local.test/server.key 2048
echo "Created private key: services/n8n.local.test/server.key"
cat services/n8n.local.test/server.key  
# This key belongs only to this service. Right now, the key doesn't have any relationship to the local CA you're running.
# It's just an RSA-2048 private key, the same kind that you might use for an SSH connection.

### 3.2 Create the OpenSSL config (SAN‑first)

Create `server.cnf` inside `services/n8n.local.test`: This defines the configuration of the service's certificate qualities. Whatever goes into this file, will be fed into the Certificate Signing Request (CSR), along with the private key you generate next, and it'll help the CA understand what kind of certificate and information to mint for you.

```
[ req ]
default_bits       = 2048
prompt             = no
default_md         = sha256
req_extensions     = req_ext
distinguished_name = dn

[ dn ]
C  = US
ST = UT
L  = Local
O  = Homelab
CN = n8n.local.test

[ req_ext ]
subjectAltName = @alt_names

[ alt_names ]
DNS.1 = n8n.local.test
```

Add additional SANs only if this service truly needs them. If you aren't sure what a SAN (Subject Alternative Name) is in this context, then you probably don't need one and you'll be fine with what you're creating above.


In [ ]:
%%bash
### 3.4 Generate the Certificate Signing Request (CSR)

#Generating the CSR is essentially filling out a standard form.
# You're creating a request `server.csr`, configured per the `server.cnf` file, which is tied to the `server.key` you just generated.
# This is where the relationship between the server's private key and the server's actual certificate, begins.

#!/usr/bin/env bash
set -euo pipefail
echo "Generating CSR for n8n.local.test..."
cd ./services/n8n.local.test
openssl req -new \
  -key server.key \
  -out server.csr \
  -config server.cnf

In [ ]:
%%bash
### 3.5 Sign the CSR with your CA

# Now, the CA is going to sign off on that CSR. Take a look at the multiple arguments that are being passed in.
# This certificate will have a serial number generated by the CA, it will last for 825 days, it will use SHA256 for the signature digest,
# and it'll include the `-extensions` listed inside the `server.cnf` file.

openssl x509 -req \
  -in ./services/n8n.local.test/server.csr \
  -CA ./pki/ca/ca.crt \
  -CAkey ./pki/ca/ca.key \
  -CAcreateserial \
  -out  ./services/n8n.local.test/server.crt \
  -days 825 \
  -sha256 \
  -extensions req_ext \
  -extfile ./services/n8n.local.test/server.cnf

You now have:

`server.key` → private key for that particular server/service

`server.crt` → certificate signed by your CA, which can now be used to secure TLS traffic to/from something that server hosts

## 4) Trust the CA in Firefox

Firefox uses its own trust store. I'm using Firefox in this example, because it isn't Chrome and that makes it better by default. But the same logic applies and the menu flow is pretty much the same.

1. Settings → Privacy & Security

2. Certificates → View Certificates

3. Authorities → Import

4. Select ~/pki/ca/ca.crt

5. Check:Trust this CA to identify websites

6. Restart Firefox

## 5) Make the Hostname Resolve Locally
  If you’re not running a separate DNS service (like Pi-Hole) already, use `/etc/hosts`. That will ensure that your local system recognizes the hostnames tied to the certificate without having to talk to another machine to learn about it.

  If you're hosting the web service on the same machine as you're using to browse to the site:

  `127.0.0.1   n8n.local.test`

  If you're using a different machine on the LAN:

  `192.168.1.10   n8n.local.test`

  The IP must point to the reverse proxy (like Traefik or Nginx) if you're using one, not the app container.

  If you **aren't** using a reverse proxy or don't know what that means, then point it to the IP of the machine hosting the container, on whatever port was exposed on the host in the Docker Compose config.

## 6) Verification Toolkit (Use These Constantly)

In [ ]:
%%bash
### 6.1 Confirm a file is a private key
#!/usr/bin/env bash
set -euo pipefail
echo "Confirming private key for n8n.local.test..."
openssl rsa -in  ./services/n8n.local.test/server.key -check

In [ ]:
%%bash
### 6.2 Confirm a file is a certificate
#!/usr/bin/env bash
set -euo pipefail
echo "Confirming certificate for n8n.local.test..."
openssl x509 -in  ./services/n8n.local.test/server.crt -noout -subject -issuer

In [ ]:
%%bash
### 6.3 Confirm SANs
#!/usr/bin/env bash
set -euo pipefail
echo "Confirming SANs for n8n.local.test..."
openssl x509 -in  ./services/n8n.local.test/server.crt -noout -text | grep -A2 "Subject Alternative Name"

In [ ]:
%%bash
### 6.4 Test live TLS with SNI
#!/usr/bin/env bash
set -euo pipefail
echo "Testing live TLS connection to n8n.local.test..."
openssl s_client \
  -connect n8n.local.test:443 \
  -servername n8n.local.test

Expected from 6.4:

Subject CN = n8n.local.test

Issuer = your CA

No default Traefik cert

## 7) Common Errors and What They Mean

|Error|Meaning|
|-----|-------|
|found a certificate rather than a key|.key file contains a cert|
|Browser warning|CA not trusted or SAN mismatch|
|Works in curl, not Firefox|CA not imported into Firefox|

## 8) Rotation and Hygiene Tips

- Rotate service certs yearly **at a minimum**

- Rotate CA every 5–10 years if you're using it long term; cycle it once a year if you have shorter lifecycles (6 months or less) on your service certs

- **Never** reuse private keys; reusing even one, means you've doubled the amount of potential services any compromise can take advantage of

- **Never** share keys between services; same as the above... both private AND public keys need to be separate and created per-service

- Keep CA offline when not issuing certs
- - Ideally, keep it out of your Home directory and store it somewhere under an additional layer of encryption
- - Decrypt and mint new certs as required, put it back in the box